<a href="https://colab.research.google.com/github/NimishK26/PowerBI-RAG-Assistant/blob/main/U5663426.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PowerBI RAG Assistant

# Data Preprocessing

Chunking

This section reads all markdown pages from the Power BI docs, extracts titles and text, and splits each document into overlapping chunks.

In [ ]:
!git clone https://github.com/MicrosoftDocs/powerbi-docs.git


Cloning into 'powerbi-docs'...
remote: Enumerating objects: 350115, done.
remote: Counting objects: 100% (749/749), done.
remote: Compressing objects: 100% (263/263), done.
remote: Total 350115 (delta 564), reused 587 (delta 486), pack-reused 349366 (from 3)
Receiving objects: 100% (350115/350115), 2.56 GiB | 35.84 MiB/s, done.
Resolving deltas: 100% (250272/250272), done.
Updating files: 100% (11019/11019), done.


In [ ]:
import os
import glob
import re
import pickle
from langchain.text_splitter import RecursiveCharacterTextSplitter

def md_path_to_url(md_path: str) -> str:
    rel = md_path.replace("powerbi-docs/powerbi-docs/", "").replace(".md", "")
    rel = rel.lower()
    return f"https://docs.microsoft.com/en-us/power-bi/{rel}"

md_files = glob.glob("powerbi-docs/powerbi-docs/**/*.md", recursive=True)

all_docs = []
for md_path in md_files:
    with open(md_path, "r", encoding="utf-8") as f:
        content = f.read()
    title_match = re.search(r"^# (.+)$", content, flags=re.MULTILINE)
    if title_match:
        title = title_match.group(1).strip()
    else:
        title = os.path.basename(md_path).replace(".md", "")
    url = md_path_to_url(md_path)
    all_docs.append({
        "file_path": md_path,
        "title":     title,
        "text":      content,
        "url":       url
    })

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

doc_chunks = []
for doc in all_docs:
    pieces = splitter.split_text(doc["text"])
    for piece in pieces:
        doc_chunks.append({
            "title":  doc["title"],
            "url":    doc["url"],
            "text":   piece,
            "source": "docs"
        })


# Embedding

This section loads the MPNet model on GPU and computes embeddings for every chunk.


In [ ]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

embed_model = SentenceTransformer("all-mpnet-base-v2", device=DEVICE)

texts = [chunk["text"] for chunk in doc_chunks]

batch_size = 128
emb_list = []
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i : i + batch_size]
    batch_emb = embed_model.encode(batch_texts, convert_to_numpy=True).astype("float32")
    emb_list.append(batch_emb)

embeddings = np.vstack(emb_list)


Vector Database Storage

Here we normalize embeddings (for cosine similarity) and build a FAISS index. The index and chunk metadata are then saved to disk.

In [ ]:
!pip install faiss-cpu -q

import faiss
import pickle


import numpy as np
norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
embeddings = embeddings / (norms + 1e-10)

dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

faiss.write_index(index, "powerbi_docs_index_mpnet_over100.faiss")
with open("powerbi_docs_chunks_mpnet_over100.pkl", "wb") as f:
    pickle.dump(doc_chunks, f)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 69.3 MB/s eta 0:00:00


# Retrieval & Reranking

Baseline Retrieval

We load the saved FAISS index and chunk metadata, reinitialize MPNet for query embeddings, and define a function to retrieve the top‐k unique chunks (applying simple URL heuristics).

In [ ]:
# @title
import os
import pickle
import faiss
import torch
import numpy as np
import re

from typing import List, Dict
from sentence_transformers import SentenceTransformer


assert os.path.exists("powerbi_docs_index_mpnet_over100.faiss"), \
       "powerbi_docs_index_mpnet_over100.faiss not found."
assert os.path.exists("powerbi_docs_chunks_mpnet_over100.pkl"), \
       "powerbi_docs_chunks_mpnet_over100.pkl not found."

index_mpnet = faiss.read_index("powerbi_docs_index_mpnet_over100.faiss")
with open("powerbi_docs_chunks_mpnet_over100.pkl", "rb") as f:
    all_chunks_mpnet: List[Dict] = pickle.load(f)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
embed_model = SentenceTransformer("all-mpnet-base-v2", device=DEVICE)

def retrieve_unique(query: str, k: int = 3, k_raw: int = 20) -> List[Dict]:
    q_emb = embed_model.encode([query], convert_to_numpy=True).astype("float32")
    norm = np.linalg.norm(q_emb, axis=1, keepdims=True)
    q_emb_norm = q_emb / (norm + 1e-10)

    distances, indices = index_mpnet.search(q_emb_norm, k_raw)
    distances = distances.flatten()
    indices   = indices.flatten()

    lower = query.lower()
    combined = list(zip(distances.tolist(), indices.tolist()))

    if "csv" in lower:
        combined.sort(key=lambda x: (
            0 if "desktop-text-and-csv-files" in all_chunks_mpnet[x[1]]["url"] else 1,
            x[0]
        ))
    if "duplicate" in lower:
        combined.sort(key=lambda x: (
            0 if "/desktop-shape-and-combine-data#remove-duplicate-rows" in all_chunks_mpnet[x[1]]["url"] else 1,
            x[0]
        ))
    if "format" in lower and "measure" in lower:
        combined.sort(key=lambda x: (
            0 if "/desktop-format-measures" in all_chunks_mpnet[x[1]]["url"] else 1,
            x[0]
        ))
    if "totalytd" in lower:
        combined.sort(key=lambda x: (
            0 if "/dax/totalytd-function-dax" in all_chunks_mpnet[x[1]]["url"] else 1,
            x[0]
        ))
    if "group by" in lower:
        combined.sort(key=lambda x: (
            0 if "/desktop-shape-and-combine-data#group-by" in all_chunks_mpnet[x[1]]["url"] else 1,
            x[0]
        ))
    if "rolling 12" in lower:
        combined.sort(key=lambda x: (
            0 if "datesinperiod" in all_chunks_mpnet[x[1]]["url"] else 1,
            x[0]
        ))
    if "paginated" in lower and "quarter" in lower:
        combined.sort(key=lambda x: (
            0 if ("report-design" in all_chunks_mpnet[x[1]]["url"] and "quarter" in all_chunks_mpnet[x[1]]["url"]) else 1,
            x[0]
        ))

    distances, indices = zip(*combined)
    distances = np.array(distances).astype("float32")
    indices   = np.array(indices).astype("int64")


    seen_urls = set()
    unique_results = []
    for dist, idx in zip(distances, indices):
        if idx < 0 or idx >= len(all_chunks_mpnet):
            continue
        chunk = all_chunks_mpnet[idx]
        url = chunk["url"]
        if url in seen_urls:
            continue
        seen_urls.add(url)

        text = chunk["text"]
        first_word = query.split()[0]
        match = re.search(re.escape(first_word), text, flags=re.IGNORECASE)
        if match:
            start = max(match.start() - 100, 0)
            end   = min(match.end() + 100, len(text))
            snippet = text[start:end].replace("\n", " ").strip()
        else:
            snippet = text.replace("\n", " ").strip()[:500]

        unique_results.append({
            "title":    chunk["title"],
            "url":      url,
            "distance": float(dist),
            "snippet":  snippet + " …"
        })
        if len(unique_results) == k:
            break

    return unique_results


if __name__ == "__main__":
    sample_query = "How do I create a bar chart in Power BI?"
    top_chunks = retrieve_unique(sample_query, k=3, k_raw=20)
    for i, res in enumerate(top_chunks, start=1):
        print(f"{i}. {res['url']}  (Distance: {res['distance']:.4f})")
        print(f"   Snippet: {res['snippet']}\n")


1. https://docs.microsoft.com/en-us/power-bi/developer/visuals/create-bar-chart  (Distance: 0.8991)
   Snippet: # Tutorial: Build a bar chart  This tutorial shows you how to develop a Power BI visual that displays data in the form of a simple bar chart. This vi …

2. https://docs.microsoft.com/en-us/power-bi/paginated-reports/report-design/visualizations/charts-report-builder  (Distance: 0.8621)
   Snippet: - [Tutorial: Add a Bar Chart to Your Report &#40;Power BI Report Builder&#41;](/sql/reporting-services/tutorial-add-a-bar-chart-to-your-report-report-builder) …

3. https://docs.microsoft.com/en-us/power-bi/visuals/power-bi-visualization-column-charts  (Distance: 0.8360)
   Snippet: # [Power BI service](#tab/powerbi-service)  Before we begin, if you haven't already done so, refer to the **[Prerequisites](#prerequisites)** section above, and follow the steps under the tab for **Power BI service**.  For this example, let’s create a column chart starting from the **Visualizations pane*

# Generation

Prompt Template

We load Llama-2 via CTransformers and build a simple prompt that includes only the top retrieved snippet plus the user’s question, instructing the model to answer concisely and append the URL.

In [ ]:
!pip install langchain-community ctransformers -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


Answer Function

This function calls the LLM with the prompt, then unconditionally appends the top chunk’s URL at the end of the answer.

In [ ]:
import torch
from langchain_community.llms import CTransformers

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

llm = CTransformers(
    model="TheBloke/Llama-2-7B-Chat-GGML",
    model_type="llama",
    config={
        "max_new_tokens": 128,
        "temperature": 0.1,
        "top_p": 0.8
    }
)
print("Llama-2-7B-Chat loaded")

USE_REWRITE = False

def debug_retrieve(query: str, k_raw: int = 18) -> None:
    rewritten = query
    q_emb = embed_model.encode([rewritten], convert_to_numpy=True).astype("float32")
    norm = torch.norm(torch.from_numpy(q_emb), dim=1, keepdim=True).numpy()
    q_emb_norm = q_emb / (norm + 1e-10)
    distances, indices = index_mpnet.search(q_emb_norm, k_raw)
    distances = distances.flatten()
    indices = indices.flatten()
    print(f"\nTop {k_raw} raw candidates for: '{query}'")
    for i in range(min(5, len(indices))):
        idx = indices[i]
        chunk = all_chunks_mpnet[idx]
        print(f"  {i+1}. {chunk['url']}  (Distance: {distances[i]:.3f})")
    print()

def build_simple_prompt(query: str, top_chunk: dict) -> str:
    snippet = top_chunk["snippet"]
    if len(snippet) < 100 and "text" in top_chunk:
        snippet = top_chunk["text"].replace("\n", " ").strip()[:400]
    snippet = snippet.replace("\n", " ").strip()
    lines = [
        "### System:\n",
        "You are a Power BI assistant. Use ONLY the context below to answer concisely, and then append the documentation URL at the very end.\n\n",
        "### Context:\n",
        f"{snippet}\n\n",
        "### Question:\n",
        f"{query}\n\n",
        "### Answer:\n"
    ]
    return "".join(lines)

def generate_simple_answer(query: str, top_chunk: dict) -> str:
    prompt = build_simple_prompt(query, top_chunk)
    assert len(prompt) < 16000, "Prompt too long—trim snippet or reduce k."
    response = llm(prompt).strip()
    return response + f"\n\nLink: {top_chunk['url']}"

def rag_query_simple(query: str, k: int = 3) -> None:
    debug_retrieve(query, k_raw=18)
    top_chunks = retrieve_unique(query, k=k, k_raw=18)
    top = top_chunks[0]
    print(f"Top Retrieved (post-filter): {top['url']}  (Distance: {top['distance']:.3f})")
    print(f"Snippet: {top['snippet']}\n")
    print("Generating concise answer …")
    answer = generate_simple_answer(query, top)
    print("\n--- Generated Answer ---\n")
    print(answer)
    print("\n--- End of Answer ---\n")

if __name__ == "__main__":
    rag_query_simple("Create a bar chart in Power BI Desktop", k=3)


Using device: cuda


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Llama-2-7B-Chat loaded

Top 18 raw candidates for: 'Create a bar chart in Power BI Desktop'
  1. https://docs.microsoft.com/en-us/power-bi/developer/visuals/create-bar-chart  (Distance: 0.902)
  2. https://docs.microsoft.com/en-us/power-bi/developer/visuals/create-bar-chart  (Distance: 0.899)
  3. https://docs.microsoft.com/en-us/power-bi/paginated-reports/report-design/visualizations/charts-report-builder  (Distance: 0.864)
  4. https://docs.microsoft.com/en-us/power-bi/developer/visuals/create-bar-chart  (Distance: 0.848)
  5. https://docs.microsoft.com/en-us/power-bi/visuals/power-bi-visualization-column-charts  (Distance: 0.839)

Top Retrieved (post-filter): https://docs.microsoft.com/en-us/power-bi/developer/visuals/create-bar-chart  (Distance: 0.902)
Snippet: --- title: Build a bar chart visual in Power BI  description: Learn how to build a sample Power BI visual with code using this step-by-step guide. This sample visual displays a bar chart. author: mberdugo ms.author: billmath

<ipython-input-14-62983c02a44b>:54: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt).strip()



--- Generated Answer ---

To create a bar chart in Power BI Desktop, follow these steps:

1. In the Modeling tool, click on "Bar Chart" and select it.
2. In the Data Marker, drag and drop the column(s) you want to use for the x-axis.
3. In the Values marker, drag and drop the column(s) you want to use for the y-axis.
4. Use the "Bar Chart" pane to customize the chart's appearance, such as changing the colors, adding labels, etc.
5. Preview the chart

Link: https://docs.microsoft.com/en-us/power-bi/developer/visuals/create-bar-chart

--- End of Answer ---



# Evaluation

Test Queries

We prepare ten questions—seven queries plus three more challenging ones—for end-to-end evaluation.

In [ ]:
all_queries = [
    "Create a bar chart in Power BI Desktop",
    "Create a column chart in Power BI Desktop",
    "Add a slicer to a report in Power BI Desktop",
    "Publish to Power BI service",
    "Apply filters in Power BI Desktop",
    "Rename a query in Power Query Editor",
    "Write a DAX measure for year-to-date sales using the TOTALYTD function",
    "In Power BI Desktop, how do I create a waterfall chart?",
    "In Power BI Paginated Report Builder, how do I pass a parameter for drill-through?",
    "How do I configure row-level security in Power BI Desktop?"
]


Evaluation Loop

In [ ]:

print("\n===== Running evaluation on 10 queries =====\n")

for idx, q in enumerate(all_queries, start=1):
    print(f"--- Query #{idx} ---")
    print("Question:", q)

    top_chunks = retrieve_unique(q, k=3, k_raw=20)
    top = top_chunks[0]
    print(f"Top Retrieved URL : {top['url']}  (Distance: {top['distance']:.3f})")
    print(f"Snippet            : {top['snippet'][:200]}…\n")

    answer = generate_simple_answer(q, top)
    print("--- Generated Answer ---")
    print(answer)
    print("--- End of Answer ---\n")


===== Running evaluation on 10 queries =====

--- Query #1 ---
Question: Create a bar chart in Power BI Desktop
Top Retrieved URL : https://docs.microsoft.com/en-us/power-bi/developer/visuals/create-bar-chart  (Distance: 0.902)
Snippet            : --- title: Build a bar chart visual in Power BI  description: Learn how to build a sample Power BI visual with code using this step-by-step guide. This sample visual displays a bar chart. author: mber…

--- Generated Answer ---
To create a bar chart in Power BI Desktop, follow these steps:

1. In the Modeling tab, click on "New Visual" and select "Bar Chart".
2. In the Bar Chart pane, drag the fields you want to use for the x-axis and y-axis into the respective columns.
3. In the Data pane, click on "Add Column" and select the column you want to use for the bar chart.
4. In the Visualization pane, click on "Bar Chart" and adjust the settings as needed.
5. Preview the visualization

Link: https://docs.microsoft.com/en-us/power-bi/developer/v

Test the Same Question with Multiple Paraphrases

We take “Create a bar chart in Power BI Desktop” and phrase it five different ways to test retrieval robustness and answer consistency.

In [ ]:
paraphrased_questions = [
    "Create a bar chart in Power BI Desktop",
    "How can I build a bar chart using Power BI Desktop?",
    "What steps are required to make a bar chart in Power BI Desktop?",
    "Guide me through creating a bar chart in Power BI Desktop",
    "In Power BI Desktop, how would I go about making a bar chart?"
]

print("\n===== Testing One Question in 5 Different Phrasings =====\n")

for idx, q in enumerate(paraphrased_questions, start=1):
    print(f"--- Paraphrase #{idx} ---")
    print("Question:", q)

    top_chunks = retrieve_unique(q, k=3, k_raw=10)
    top = top_chunks[0]
    print(f"Top Retrieved URL : {top['url']}  (Distance: {top['distance']:.3f})")
    print(f"Snippet            : {top['snippet'][:200]}…\n")

    answer = generate_simple_answer(q, top)
    print("--- Generated Answer ---")
    print(answer)
    print("--- End of Answer ---\n")



===== Testing One Question in 5 Different Phrasings =====

--- Paraphrase #1 ---
Question: Create a bar chart in Power BI Desktop
Top Retrieved URL : https://docs.microsoft.com/en-us/power-bi/developer/visuals/create-bar-chart  (Distance: 0.902)
Snippet            : --- title: Build a bar chart visual in Power BI  description: Learn how to build a sample Power BI visual with code using this step-by-step guide. This sample visual displays a bar chart. author: mber…

--- Generated Answer ---
To create a bar chart in Power BI Desktop, follow these steps:

1. In the Modeling tab, click on "New Visual" and select "Bar Chart".
2. In the Bar Chart pane, drag the fields you want to use for the x-axis and y-axis into the respective columns.
3. In the Data pane, select the table or dataset you want to visualize.
4. Click on the "Model" button in the Bar Chart pane and select "Bar Chart".
5. In the Bar Chart dialog box, adjust the settings as needed, such

Link: https://docs.microsoft.com/en-us/